In [1]:
import os
os.chdir("../module/")

import ecc_editor as e_ed
import ecc_library as e_lib
import interference_and_attacks as iaa
import secret_block as sb
import stat_treak as stat 

os.chdir("../notepads/data/")

### Глабальные парамерты системы

In [2]:
#p_erorr_global = 0.03
p_erorr_global = 0.05
key = b"123_mail_UP"

### Блок статистики (stst_treak)

In [3]:
name_cod_box = stat.selection_g_x(p_erorr_global)
print("Вероятность естественной ошибки",p_erorr_global)
name_cod_box

Вероятность естественной ошибки 0.05


('63_7_15', 95457791829701599)

In [4]:
p_sintetik = stat.selection_sintetik_p(p_erorr_global,name_cod_box[0])
print("Вероятность искуственной ошибки",p_sintetik)
name_cod_mail = stat.selection_g_x(p_erorr_global,0)
name_cod_mail

Вероятность искуственной ошибки 0.06620442158513286


('7_1_3', 127)

### Блоки кодирования (ecc_editor)

In [5]:
treak_to_box = "long_text.txt"
box_editer = e_ed.ecc_editor(name_cod_box[0])
long_box = box_editer.cod_work(treak_to_box)
len_box = len(long_box)
#long_box[:127]

In [6]:
len_box * p_sintetik

59455.87387085234

In [7]:
treak_to_mail = "mail.txt"
mail_editer = e_ed.ecc_editor(name_cod_mail[0])
long_mail = mail_editer.cod_work(treak_to_mail)
long_mail[:127]

array([0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
       1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1.])

In [8]:
print("Ожидаеммая длинна контенера",len_box * p_sintetik)
print("Реальная длинна контейнера",len(long_mail))

Ожидаеммая длинна контенера 59455.87387085234
Реальная длинна контейнера 57680


### Размещение сообщения (secret_block)

In [9]:
sicret_ed_in = sb.secret_editor(key)
to_link = sicret_ed_in.merge_sequences(long_box,long_mail,p_sintetik)
to_link[:100]

array([0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0])

In [10]:
print(f"Вероятность искуственной ошибки {sum(to_link != long_box) / len(long_box)}")

Вероятность искуственной ошибки 0.03285619637776776


### Ошибки в линии связи (interference_and_attacks)

In [121]:
dop = 1
from_link = iaa.noise_in_treak(to_link,p_erorr_global * dop)
print(f"Вероятность естественной ошибки {sum(to_link != from_link) / len(from_link)}")

Вероятность естественной ошибки 0.10005066448419658


In [122]:
print(f"Полная вероятность в кейсе {sum(long_box != from_link) / len(from_link)}")

Полная вероятность в кейсе 0.1263282724524394


### Бок разделения сообщений

In [123]:
name_cod_box_end = stat.selection_g_x(p_erorr_global)
name_cod_box_end

('63_7_15', 95457791829701599)

In [124]:
p_sintetik_end = stat.selection_sintetik_p(p_erorr_global,name_cod_box[0])
name_cod_mail_end = stat.selection_g_x(p_erorr_global,0)
name_cod_mail_end

('7_1_3', 127)

In [125]:
sicret_ed_out = sb.secret_editor(key)
long_mail_end = sicret_ed_out.separation_sequences(from_link,p_sintetik_end)

cut_point = sicret_ed_out.gen_mark_sequence_bin(int(name_cod_mail_end[0].split("_")[0]))

### Блок декодирования

In [126]:
out_box_e = e_ed.ecc_editor(name_cod_box_end[0])
box_end = out_box_e.decoding_work(from_link)

In [127]:
box_out = e_ed.FailBin("box_out.txt")
box_out.write_bin(box_end)

In [128]:
out_mail_e = e_ed.ecc_editor(name_cod_mail_end[0])
mail_end = out_mail_e.decoding_work(long_mail_end,cut_point,True)

In [129]:
box_mail = e_ed.FailBin("mail_out.txt")
box_mail.write_bin(mail_end)

### Оценка качества

In [130]:
box_et = e_ed.FailBin("long_text.txt")
box_et_mas = box_et.read_bin()
box_et_mas

array([0, 1, 0, ..., 0, 1, 0])

In [131]:
box_test = e_ed.FailBin("box_out.txt")
box_test_mas = box_test.read_bin()
box_test_mas

array([0, 1, 0, ..., 0, 1, 0])

In [132]:
sum(box_et_mas == box_test_mas) / len(box_et_mas)  * 100

99.83364066383388

In [133]:
mail_et = e_ed.FailBin("mail.txt")
mail_et_mas = mail_et.read_bin()

mail_test = e_ed.FailBin("mail_out.txt")
mail_test_mas = mail_test.read_bin()

print(len(mail_test_mas))
print(len(mail_et_mas))
sum(mail_et_mas[:4704] == mail_test_mas[:8240]) / len(mail_et_mas[:4704]) * 100

2048
8240


<ipython-input-133-d87f5c2805ca>:9: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  sum(mail_et_mas[:4704] == mail_test_mas[:8240]) / len(mail_et_mas[:4704]) * 100


TypeError: 'bool' object is not iterable

In [ ]:
t = [1]
t2 = []
t3 = []